[IPython Notebook](alternate_backends.ipynb) |  [Python Script](alternate_backends.py)

Advanced: Alternate Backends
============================

Setup
-----------------------------

As always, let's do imports and initialize a logger and a new Bundle.  See [Building a System](building_a_system.html) for more details.

In [1]:
import phoebe
from phoebe import u # units
import numpy as np
import matplotlib.pyplot as plt

logger = phoebe.logger(clevel='INFO')

b = phoebe.Bundle.default_binary()

/usr/local/lib/python2.7/dist-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)
/usr/local/lib/python2.7/dist-packages/astropy/units/quantity.py:732: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  return super(Quantity, self).__eq__(other)


And we'll attach some dummy datasets.  See [Datasets](datasets.html) for more details.

In [2]:
b.add_dataset('ORB', time=np.linspace(0,10,1000), dataset='orb01', component=['primary', 'secondary'])

<ParameterSet: 3 parameters | components: _default, primary, secondary>

In [3]:
b.add_dataset('LC', time=np.linspace(0,10,1000), dataset='lc01')

<ParameterSet: 18 parameters | methods: LC, LC_dep>

Available Backends
----------------------------

See the [Compute Tutorial](./compute) for details on adding compute options and using them to create synthetic models.


### PHOEBE 1.0 Legacy

For more details, see [Comparing PHOEBE 2.0 vs PHOEBE Legacy](../examples/legacy)

In [4]:
b.add_compute('legacy', compute='legacycompute')
print b['legacycompute']

store_mesh@legacycompute@compute: False
atm@_default@legacycompute@compute: kurucz
morphology@legacycompute@compute: Detached binary
cindex@legacycompute@compute: False
heating@legacycompute@compute: True
gridsize@_default@legacycompute@compute: 40
mult_refl@legacycompute@compute: False
ie@legacycompute@compute: False
rv_method@legacycompute@compute: flux-weighted
enabled@lc01@legacycompute@compute: True
atm@primary@legacycompute@compute: kurucz
atm@secondary@legacycompute@compute: kurucz
gridsize@primary@legacycompute@compute: 40
gridsize@secondary@legacycompute@compute: 40


### Photodynam

For more details, see [Comparing PHOEBE 2.0 vs Photodynam (Binary)](../examples/photodynam) and [Comparing PHOEBE 2.0 vs Photodynam (Hierarchical Triple)](../examples/hierarchical_triple)

In [5]:
b.add_compute('photodynam', compute='photodynamcompute')
print b['photodynamcompute']

stepsize@photodynamcompute@compute: 0.01
orbiterror@photodynamcompute@compute: 1e-20
enabled@lc01@photodynamcompute@compute: True
enabled@orb01@photodynamcompute@compute: True


### JKTEBOP

For more details, see [Comparing PHOEBE 2.0 vs JKTEBOP](../examples/jktebop)

In [6]:
b.add_compute('jktebop', compute='jktebopcompute')
print b['jktebopcompute']

ringsize@jktebopcompute@compute: 5.0 deg
enabled@lc01@jktebopcompute@compute: True


Using Alternate Backends
---------------------------------


### Adding Compute Options

Adding a set of compute options for an alternate backend is just as easy as for the PHOEBE backend.  Simply provide the function or name of the function in phoebe.parameters.compute that points to the parameters for that backend.

Here we'll add the default PHOEBE backend as well as photodynam by Josh Carter.  Note that in order to use an alternate backend, that backend must be installed on your machine.

In [7]:
b.add_compute('phoebe', compute='phoebebackend')

<ParameterSet: 18 parameters | qualifiers: dynamics_method, gridsize, stepsize, lc_method, atm, enabled, ltte, rv_grav, orbiterror, rv_method, etv_method, maxpoints, delta, etv_tol, store_mesh, distortion_method, eclipse_alg, mesh_method>

In [8]:
print b['phoebebackend']

dynamics_method@phoebebackend@compute: keplerian
ltte@phoebebackend@compute: False
store_mesh@phoebebackend@compute: False
mesh_method@_default@phoebebackend@compute: marching
delta@_default@phoebebackend@compute: 0.1
maxpoints@_default@phoebebackend@compute: 100000
distortion_method@_default@phoebebackend@compute: roche
eclipse_alg@phoebebackend@compute: visible_ratio
atm@_default@phoebebackend@compute: blackbody
lc_method@_default@_default@phoebebackend@phoebe@compute: numerical
rv_method@phoebebackend@compute: flux-weighted
rv_grav@phoebebackend@compute: False
etv_method@phoebebackend@compute: crossing
etv_tol@phoebebackend@compute: 0.0001 d
enabled@lc01@phoebebackend@compute: True
enabled@orb01@phoebebackend@compute: True
mesh_method@primary@phoebebackend@compute: marching
mesh_method@secondary@phoebebackend@compute: marching
delta@primary@phoebebackend@compute: 0.1
delta@secondary@phoebebackend@compute: 0.1
maxpoints@primary@phoebebackend@compute: 100000
maxpoints@secondary@phoebe

In [9]:
b.add_compute('photodynam', compute='pdbackend')

<ParameterSet: 3 parameters | qualifiers: orbiterror, stepsize, enabled>

In [10]:
print b['pdbackend']

stepsize@pdbackend@compute: 0.01
orbiterror@pdbackend@compute: 1e-20
enabled@lc01@pdbackend@compute: True
enabled@orb01@pdbackend@compute: True


### Running Compute

Nothing changes for running compute - simply provide the compute tag for those options.  Do note, however, that not all backends support all dataset types.

In [11]:
b.set_value_all('pbscale', 'pblum')
# TODO: remove this once other backends support coupled pblums

In [12]:
b.run_compute('pdbackend', model='pdresults')

<ParameterSet: 16 parameters | methods: ORB, LC>

### Running Fitting

COMING SOMEDAY

Running Multiple Backends Simultaneously
---------------------------------------------------

Running multiple backends simultaneously is just as simple as running the PHOEBE backend with multiple sets of compute options (see [Compute](compute.html)).

We just need to make sure that each dataset is only enabled for one (or none) of the backends that we want to use, and then send a list of the compute tags to run_compute.  Here we'll use the PHOEBE backend to compute orbits and the photodynam backend to compute light curves.

In [13]:
b.set_value_all('enabled@lc01@phoebebackend', False)
b.set_value_all('enabled@orb01@pdbackend', False)
print b['enabled']

enabled@lc01@legacycompute@compute: True
enabled@lc01@photodynamcompute@compute: True
enabled@orb01@photodynamcompute@compute: True
enabled@lc01@jktebopcompute@compute: True
enabled@lc01@phoebebackend@compute: False
enabled@orb01@phoebebackend@compute: True
enabled@lc01@pdbackend@compute: True
enabled@orb01@pdbackend@compute: False


In [14]:
b.run_compute(['phoebebackend', 'pdbackend'], model='mixedresults')

<ParameterSet: 16 parameters | methods: ORB, LC>

The parameters inside the returned model even remember which set of compute options (and therefore, in this case, which backend) were used to compute them.

In [15]:
print b['mixedresults'].computes

['phoebebackend', 'pdbackend']


In [16]:
b['mixedresults@phoebebackend'].datasets

['orb01']

In [17]:
b['mixedresults@pdbackend'].datasets

['lc01']